##Initial configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

os.chdir(f'/content/drive/MyDrive/Polimi/NLP/project/t5_generative_model')
os.getcwd()

'/content/drive/MyDrive/Polimi/NLP/project/t5_generative_model'

In [ ]:
#!pip install --upgrade numpy
#!pip install --upgrade gensim

##Data analysis

###Download of the tools

In [ ]:
#!pip uninstall transformers -y

In [ ]:
#! pip install transformers

###Load of the dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

Import of all the libraries

In [ ]:
import torch
import pandas as pd
from transformers import T5Tokenizer
from datasets import load_dataset
from datasets import Dataset
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback, pipeline
from transformers import TrainerCallback
from functools import reduce
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering

2025-05-14 09:11:51.983371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747213912.172132      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747213912.224747      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Here we load the training and test set

In [ ]:
#!pip install --upgrade datasets fsspec
#!rm -rf ~/.cache/huggingface/datasets/neural-bridge/rag-dataset-12000

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

In [ ]:
# Load the dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

(…)-00000-of-00001-9df3a936e1f63191.parquet:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

(…)-00000-of-00001-af2a9f454ad1b8a3.parquet:   0%|          | 0.00/5.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
train_set = dataset["train"]
test_set = dataset["test"]

df_train = pd.DataFrame(train_set)
df_test = pd.DataFrame(test_set)

# Remove records with null values or empty/whitespace strings
columns_to_check = ["question", "context", "answer"]

# Drop rows with NaN values
df_train = df_train.dropna(subset=columns_to_check)

# Drop rows with empty or whitespace-only strings
for col in columns_to_check:
    df_train = df_train[df_train[col].str.strip() != ""]

# Reset the index
df_train = df_train.reset_index(drop=True)

##Generative question answering model with context

We will use T5-base as a generative model.

###T5

####Tokenizer

First we have to tokenize the dataset and split into training and validation set

In [ ]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocessing function
def preprocess_function(example):
    input_text = "question: " + example["question"] + " context: " + example["context"]
    target_text = example["answer"]

    # Tokenize input (truncate and pad to max length)
    model_inputs = tokenizer(
        input_text,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    # Tokenize target/label (truncate and pad to max length)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text,
            max_length=64,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Split the dataset into train and validation
train_dataset, val_dataset = train_test_split(df_train, test_size=0.1)

# Convert your pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=False)
val_dataset = val_dataset.map(preprocess_function, batched=False)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/8638 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/960 [00:00<?, ? examples/s]

####Training and testing of T5 model

#####Training

In this section we train the T5 model on our task. We used the smallet T5 model just due to performance issues.

In [ ]:
# Load the smallest model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5_qa_model",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=500,
    report_to="none",
    metric_for_best_model="eval_loss",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

/tmp/ipykernel_35/105569557.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.530900
1000,0.537300
1500,0.610300
2000,0.590800
2500,0.560300
3000,0.541700


TrainOutput(global_step=3240, training_loss=0.5598013089026934, metrics={'train_runtime': 633.8723, 'train_samples_per_second': 40.882, 'train_steps_per_second': 5.111, 'total_flos': 3507247443345408.0, 'train_loss': 0.5598013089026934, 'epoch': 3.0})

Now we save the model

In [ ]:
torch.save(model.state_dict(), 'model_T5_weights.pth')

#####Testing

We load the model

In [ ]:
model.load_state_dict(torch.load('model_T5_weights.pth'))

We use the test set to generate the answer and see how much good the model is.

In [ ]:
# The model must be in evaluation mode
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Create a copy of the first n elements of the test set
n = 200
df_test_sample = df_test.head(n).copy()

# Generate the answer
df_test_sample["generated_answer"] = df_test_sample.apply(
    lambda row: generate_answer(row["question"], row["context"]),
    axis=1
)


In [ ]:
# Loop through the first m rows in the test set and print the output in the desired format
m = 20
for idx, row in df_test_sample.head(m).iterrows():
    print(f"{idx + 1}")
    print(f"Question: {row['question']}")
    print(f"Target Answer: {row['answer']}")
    print(f"Generated Answer: {row['generated_answer']}")
    print("\n" + "-"*50 + "\n")

1
Question: Who is the music director of the Quebec Symphony Orchestra?
Target Answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.
Generated Answer: Fabien Gabel is the music director of the Quebec Symphony Orchestra.

--------------------------------------------------

2
Question: Who were the four students of the University of Port Harcourt that were allegedly murdered?
Target Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.
Generated Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.

--------------------------------------------------

3
Question: What did Paul Wall offer to all U.S. Olympic Medalists?
Target Answer: Paul Wall wants to give free gold grills to all U.S. Olympic Medalists.
Generated Answer: Paul Wall off

####T5 without any training

Now let's upload the T5 model again and test how good is the model alone without any training on our dataset

In [ ]:
model2 = T5ForConditionalGeneration.from_pretrained("t5-small")

# The model must be in evaluation mode
model2.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output_ids = model2.generate(
            input_ids,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Create a copy of the first n elements of the test set
n = 10
df_test_sample2 = df_test.head(n).copy()

# Generate the answer
df_test_sample2["generated_answer"] = df_test_sample2.apply(
    lambda row: generate_answer(row["question"], row["context"]),
    axis=1
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Here there are the answer the model would give

In [ ]:
# Loop through the first 10 rows in the test set and print the output in the desired format
for idx, row in df_test_sample2.iterrows():
    print(f"{idx + 1}")
    print(f"Question: {row['question']}")
    print(f"Target Answer: {row['answer']}")
    print(f"Generated Answer: {row['generated_answer']}")
    print("\n" + "-"*50 + "\n")

1
Question: Who is the music director of the Quebec Symphony Orchestra?
Target Answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.
Generated Answer: Fabien Gabel

--------------------------------------------------

2
Question: Who were the four students of the University of Port Harcourt that were allegedly murdered?
Target Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.
Generated Answer: Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku

--------------------------------------------------

3
Question: What did Paul Wall offer to all U.S. Olympic Medalists?
Target Answer: Paul Wall wants to give free gold grills to all U.S. Olympic Medalists.
Generated Answer: Free Gold Grills

--------------------------------------------------

4
Question: What are the main agricultural products that African countries export to the rest of the world

###Roberta

Here we use Roberta model to perform the same task of previous T5 model

####Preparation for training

Split of the dataset

In [ ]:
df_train_split, df_val_split = train_test_split(df_train, test_size=0.1, random_state=42)

We convert the two pandas DataFrames (df_train_split and df_val_split) into the format required by Hugging Face's Dataset object for question answering tasks.

In [ ]:
def convert_to_hf_format(df):
    return {
        "id": [str(i) for i in range(len(df))],
        "context": df["context"].tolist(),
        "question": df["question"].tolist(),
        "answers": [
            {
                "text": [ans],
                "answer_start": [ctx.find(ans)] if ctx.find(ans) != -1 else [0]
            }
            for ans, ctx in zip(df["answer"], df["context"])
        ]
    }

train_dataset = Dataset.from_dict(convert_to_hf_format(df_train_split))
val_dataset = Dataset.from_dict(convert_to_hf_format(df_val_split))

We build the tokenizer

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        answer_start = answers["answer_start"][0]
        answer_text = answers["text"][0]

        start_char = answer_start
        end_char = answer_start + len(answer_text)

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        if not (start_char <= offsets[token_end_index][1] and end_char >= offsets[token_start_index][0]):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    return tokenized_examples

tokenized_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(prepare_train_features, batched=True, remove_columns=val_dataset.column_names)


Map:   0%|          | 0/8638 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

The callback to print the loss during the training

In [ ]:
class LossPrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"Step {state.global_step}: loss = {logs['loss']:.4f}")

####Training

In [ ]:
model = RobertaForQuestionAnswering.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    num_train_epochs=2,
    save_strategy="epoch",              # Save at the end of each epoch
    logging_strategy="steps",           # Log every few steps
    logging_steps=250,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    callbacks=[LossPrinterCallback()]   # Optional: to print custom loss log
)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_35/3799886651.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.921200,0.903054
2,0.801600,0.860071


Step 250: loss = 1.3376
Step 500: loss = 1.0300
Step 750: loss = 1.0424
Step 1000: loss = 0.9688
Step 1250: loss = 0.9370
Step 1500: loss = 0.9378
Step 1750: loss = 0.9212
Step 2000: loss = 0.8786
Step 2250: loss = 0.7992
Step 2500: loss = 0.8025
Step 2750: loss = 0.8040
Step 3000: loss = 0.7744
Step 3250: loss = 0.8137
Step 3500: loss = 0.8016


TrainOutput(global_step=3574, training_loss=0.9145674991020625, metrics={'train_runtime': 2647.8583, 'train_samples_per_second': 21.596, 'train_steps_per_second': 1.35, 'total_flos': 1.1206495302893568e+16, 'train_loss': 0.9145674991020625, 'epoch': 2.0})

In [ ]:
torch.save(model.state_dict(), 'model_Roberta_weights.pth')

####Testing

In [ ]:
model.load_state_dict(torch.load('model_Roberta_weights.pth'))

In [ ]:
# Load your trained model
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

# Select the first 20 samples from the test set
df_test_sample = df_test.head(20)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df_test_sample[["question", "context"]])

# Apply the pipeline in batch
results = qa_pipeline(dataset)

df_test_sample["predicted_answer"] = [r["answer"] if isinstance(r, dict) else "Error" for r in results]

df_test_sample = df_test_sample.rename(columns={"answer": "true_answer"})

# Ora df_test_sample ha: question, context, true_answer, predicted_answer
df_results = df_test_sample[["question", "context", "true_answer", "predicted_answer"]]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
/tmp/ipykernel_35/778495404.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_sample["predicted_answer"] = [r["answer"] if isinstance(r, dict) else "Error" for r in results]


In [ ]:
for i, row in df_results.iterrows():
    print(f"\nExample {i+1}")
    print(f"Question: {row['question']}")
    print(f"True Answer: {row['true_answer']}")
    print(f"Predicted Answer: {row['predicted_answer']}")


Example 1
Question: Who is the music director of the Quebec Symphony Orchestra?
True Answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.
Predicted Answer: Fabien Gabel, music director of the Quebec Symphony Orchestra

Example 2
Question: Who were the four students of the University of Port Harcourt that were allegedly murdered?
True Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.
Predicted Answer: Port Harcourt – The Rivers police command on Wednesday arraigned 13 suspects before

Example 3
Question: What did Paul Wall offer to all U.S. Olympic Medalists?
True Answer: Paul Wall wants to give free gold grills to all U.S. Olympic Medalists.
Predicted Answer: Channels
Music
Style
Pop Culture
Sports
Sneakers

Example 4
Question: What are the main agricultural products that African countries export to the rest of the world?
True Answer: Africa